In [2]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
#database : dogs and cats images
print(os.listdir("../input/dogs-cats-images/dataset/training_set"))

['cats', 'dogs']


In [3]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
data_generator = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )

image_size = 224
batch_size = 10
traindata = data_generator.flow_from_directory(directory="../input/dogs-cats-images/dataset/training_set",target_size=(image_size ,image_size ),  batch_size=batch_size,)
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="../input/dogs-cats-images/dataset/test_set", target_size=(image_size,image_size))

num_classes = len(traindata.class_indices)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
model = Sequential()

model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_2 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
__________

In [ ]:
count = sum([len(files) for r, d, files in os.walk("../input/dogs-cats-images/dataset/test_set")])
print(count)
import time
start_time = time.time()
model.fit_generator(
      traindata,
        steps_per_epoch=int(count/batch_size) + 1,
        epochs=10)
print("Total time is second ~s ", time.time()-start_time)

2000
Epoch 1/10
 99/201 [=============>................] - ETA: 26s - loss: 1.2266 - acc: 0.8354

In [ ]:
from IPython.display import Image, display

import os, random
img_locations = []
for d in os.listdir("../input/dogs-cats-images/dataset/test_set/"):
    directory = "../input/dogs-cats-images/dataset/test_set/" + d
    sample = [directory + '/' + s for s in random.sample(
        os.listdir(directory), int(random.random()*10))]
    img_locations += sample
print("Test data found")    

In [ ]:
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    return preprocess_input(img_array)

random.shuffle(img_locations)
imgs = read_and_prep_images(img_locations)
predictions = model.predict_classes(imgs)
classes = dict((v,k) for k,v in traindata.class_indices.items())
print("Dogs and cats classification : ")
for img, prediction in zip(img_locations, predictions):
    display(Image(img))
    print(classes[prediction])